In [50]:
import websocket, json, pprint, talib, numpy
from binance.client import Client
from binance.enums import *
import config
import numpy as np
import pandas as pd

In [5]:
SOCKET = "wss://stream.binance.com:9443/ws/ethusdt@kline_1m"

RSI_PERIOD = 14
RSI_OVERBOUGHT = 70
RSI_OVERSOLD = 30
TRADE_SYMBOL = 'ETHUSD'
TRADE_QUANTITY = 0.05

closes = []
in_position = False

client = Client(config.KEY, config.SECRET, tld='us')

def order(side, quantity, symbol,order_type=ORDER_TYPE_MARKET):
    try:
        print("sending order")
        order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
        print(order)
    except Exception as e:
        print("an exception occured - {}".format(e))
        return False

    return True

    
def on_open(ws):
    print('opened connection')

def on_close(ws):
    print('closed connection')

def on_message(ws, message):
    global closes, in_position
    
    print('received message')
    json_message = json.loads(message)
    pprint.pprint(json_message)

    candle = json_message['k']

    is_candle_closed = candle['x']
    close = candle['c']

    if is_candle_closed:
        print("candle closed at {}".format(close))
        closes.append(float(close))
        print("closes")
        print(closes)

        if len(closes) > RSI_PERIOD:
            np_closes = numpy.array(closes)
            rsi = talib.RSI(np_closes, RSI_PERIOD)
            print("all rsis calculated so far")
            print(rsi)
            last_rsi = rsi[-1]
            print("the current rsi is {}".format(last_rsi))

            if last_rsi > RSI_OVERBOUGHT:
                if in_position:
                    print("Overbought! Sell! Sell! Sell!")
                    # put binance sell logic here
                    order_succeeded = order(SIDE_SELL, TRADE_QUANTITY, TRADE_SYMBOL)
                    if order_succeeded:
                        in_position = False
                else:
                    print("It is overbought, but we don't own any. Nothing to do.")
            
            if last_rsi < RSI_OVERSOLD:
                if in_position:
                    print("It is oversold, but you already own it, nothing to do.")
                else:
                    print("Oversold! Buy! Buy! Buy!")
                    # put binance buy order logic here
                    order_succeeded = order(SIDE_BUY, TRADE_QUANTITY, TRADE_SYMBOL)
                    if order_succeeded:
                        in_position = True

                
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
#ws.run_forever()

In [69]:
endTime = 1633060800000
interval = 675000000
i = 0
candles = []

client = Client(config.KEY, config.SECRET, tld='us')

while(len(candles) < 15000):
    currCandles = client.get_klines(symbol='ETHUSDT', 
                                interval=Client.KLINE_INTERVAL_15MINUTE, 
                                startTime = (endTime - interval), 
                                endTime = (endTime),
                                limit = 750)

    
    closes = []
    volumes = []
    
    print(len(currCandles))
    
    for i in range(len(currCandles)):
        candle = currCandles[i]
        close = float(candle[4])
        volume = float(candle[5])

        if(len(closes) == 15):
            obv = talib.OBV(np.array(closes), np.array(volumes))
            rsi = talib.RSI(np.array(closes), 14)
            candle.append(rsi[-1])
            candle.append(obv[-1])
            candles.append(candle)
            closes = closes[:-1]
            volumes = volumes[:-1]

        closes.insert(0, close)
        volumes.insert(0, volume)
        
        i+=1 
        #print(i)
    print("YES", len(candles))
    endTime += interval
print('\n\n\nFINAL\n\n', candles)

750
YES 735
750
YES 1470
750
YES 2205
750
YES 2940
750
YES 3675
750
YES 4410
750
YES 5145
750
YES 5880
750
YES 6615
750
YES 7350
750
YES 8085
750
YES 8820
750
YES 9555
750
YES 10290
750
YES 11025
750
YES 11760
750
YES 12495
750
YES 13230
750
YES 13965
750


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [70]:
print(len(candles))

15435


In [71]:
opens = []
closes = []
highs = []
lows = []
volumes = []
qavs = []
numTrades = []
tbbavs = []
tbqavs = []
rsis = []
obvs = []
labels = []



for i in range(len(candles) - 1):
    c = candles[i]
    
    opens.append(c[1])
    highs.append(c[2])
    lows.append(c[3])
    closes.append(c[4])
    volumes.append(c[5])
    qavs.append(c[7])
    numTrades.append(c[8])
    tbbavs.append(c[9])
    tbqavs.append(c[10])
    rsis.append(c[12])
    obvs.append(c[13])
    labels.append(1 if (candles[i+1][4] > c[4]) else 0)

In [75]:
df = pd.DataFrame(data={"opens": opens, "highs": highs, "lows": lows, "closes": closes, "volumes": volumes, "qavs": qavs, "numTrades": numTrades, "tbbavs": tbbavs, "tbqavs": tbqavs, "rsis": rsis,"obvs": obvs})
df.head()

,opens,highs,lows,closes,volumes,qavs,numTrades,tbbavs,tbqavs,rsis,obvs
0,3058.10000000,3078.17000000,3056.55000000,3072.52000000,54.66974000,167664.99937970,156,29.33605000,89988.05769240,76.031607,45.85872
1,3074.51000000,3090.71000000,3067.57000000,3084.20000000,99.57095000,306721.47637890,145,32.32306000,99472.95267810,65.670788,-28.24838
2,3084.20000000,3084.20000000,3067.57000000,3070.14000000,56.38186000,173264.96292150,121,21.55573000,66189.62437540,64.727017,14.10146
3,3068.42000000,3076.16000000,3061.76000000,3069.71000000,46.51578000,142791.79711720,92,14.83439000,45533.23755210,67.015810,-0.58536
4,3076.19000000,3081.94000000,3070.63000000,3077.14000000,66.24285000,203781.47787350,115,39.45869000,121354.46494090,66.628899,7.86702


In [79]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled, columns = df.columns)
scaled_df['labels'] = labels
scaled_df.head()


,opens,highs,lows,closes,volumes,qavs,numTrades,tbbavs,tbqavs,rsis,obvs,labels
0,0.329456,0.330893,0.334030,0.334446,0.026131,0.017676,0.034536,0.015266,0.010333,0.771987,0.477127,1
1,0.335601,0.335586,0.338136,0.338821,0.047593,0.032336,0.032101,0.016821,0.011422,0.661111,0.463088,0
2,0.339230,0.333150,0.338136,0.333554,0.026950,0.018266,0.026788,0.011218,0.007601,0.651011,0.471111,0
3,0.333321,0.330141,0.335971,0.333393,0.022234,0.015054,0.020368,0.007720,0.005229,0.675504,0.468329,1
4,0.336231,0.332304,0.339276,0.336176,0.031663,0.021483,0.025459,0.020534,0.013935,0.671364,0.469930,1


In [80]:
df.to_csv("./data.csv", sep=',',index=False)

In [73]:
data = pd.read_csv('data.csv')
data.head()

,opens,highs,lows,closes,volumes,qavs,numTrades,tbbavs,tbqavs,rsis,obvs,labels
0,3058.10,3078.17,3056.55,3072.52,54.66974,167664.999380,156,29.33605,89988.057692,76.031607,45.85872,1
1,3074.51,3090.71,3067.57,3084.20,99.57095,306721.476379,145,32.32306,99472.952678,65.670788,-28.24838,0
2,3084.20,3084.20,3067.57,3070.14,56.38186,173264.962922,121,21.55573,66189.624375,64.727017,14.10146,0
3,3068.42,3076.16,3061.76,3069.71,46.51578,142791.797117,92,14.83439,45533.237552,67.015810,-0.58536,1
4,3076.19,3081.94,3070.63,3077.14,66.24285,203781.477873,115,39.45869,121354.464941,66.628899,7.86702,1
